Setup an existing AzureML Workspace. To create a new Workspace follow instructions at: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-workspace?tabs=azure-portal#create-a-workspace

In [ ]:
!pip install azure-ai-ml

In [76]:
import datetime, time

from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    BatchDeployment,
    Model,
    Environment,
    BuildContext,
    CodeConfiguration,
    OnlineRequestSettings,
    ProbeSettings
)
from azure.core.exceptions import ResourceNotFoundError
from azure.identity import DefaultAzureCredential

In [2]:
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential)

Found the config file in: .\config.json
Class RegistryOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [3]:
# The below code downloads from an unofficial source.
# You can also download the model manually from the official source: https://huggingface.co/CompVis/stable-diffusion-v-1-4-original

import requests

model_download_url = "https://drive.yerf.org/wl/?id=EBfTrmcCCUAGaQBXVIj5lJmEhjoP1tgl&mode=grid&download=1"
model_file_path = "./stable-diffusion-v1.ckpt"

response = requests.get(model_download_url)
response.raise_for_status()
with open(model_file_path, "wb") as f:
    f.write(response.content)

In [39]:
model_name = "stable-diffusion-v1"

try:
    model = ml_client.models.get(model_name, version="1")
except ResourceNotFoundError:
    model = Model(
        path=model_file_path,
        name=model_name,
    )
    model = ml_client.create_or_update(model)

In [16]:
!(git submodule init && git submodule update)

In [44]:
# TODO: build=BuildContext(path="./stable-diffusion-api"),

environment_name = "stable-diffusion-v1-new"

try:
    env = ml_client.environments.get(environment_name, version="1")
except ResourceNotFoundError:
    env = Environment(
        name=environment_name,
        image="docker.io/shivammg/stable-diffusion:v1",
        inference_config={
            "liveness_route": {"port": 5000, "path": "/api/health"},
            "readiness_route": {"port": 5000, "path": "/api/health"},
            "scoring_route": {"port": 5000, "path": "/api/txt2img"},
        },
    )
    env = ml_client.environments.create_or_update(env)

In [37]:
endpoint_name_prefix = "stable-diffusion-"
endpoint_name = None
endpoint = None

for e in ml_client.online_endpoints.list():
    if e.name.startswith(endpoint_name_prefix):
        endpoint = e
        endpoint_name = e.name
        break

if not endpoint:
    endpoint_name = endpoint_name_prefix + datetime.datetime.now().strftime("%Y%m%d%H%M")  # Endpoint name needs to be unique within a region
    endpoint = ManagedOnlineEndpoint(
        name=endpoint_name,
        auth_mode="key",
    )
    ml_client.online_endpoints.begin_create_or_update(endpoint)

In [48]:
for _ in range(100):
    endpoint = ml_client.online_endpoints.get(endpoint_name)
    if endpoint.provisioning_state == 'Succeeded':
        break
    time.sleep(10)

In [49]:
deployment_name = "stable-diffusion-deployment"
deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    environment_variables={
        "MODEL_DIR": f"/var/azureml-app/azureml-models/{model_name}/1",
    },
    instance_type="Standard_NC6s_v3",
    instance_count=1,
    request_settings=OnlineRequestSettings(
        request_timeout_ms=60*1000,
        max_concurrent_requests_per_instance=2,
        max_queue_wait_ms=1*1000,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=30,
        timeout=10,
        period=20,
        initial_delay=20,
    ),
    liveness_probe=ProbeSettings(
        failure_threshold=15,
        timeout=5,
        period=10,
        initial_delay=10,
    ),
)
ml_client.online_deployments.begin_create_or_update(deployment)

Check: endpoint stable-diffusion-202301061139 exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


..........................................

In [56]:
for _ in range(100):
    deployment = ml_client.online_deployments.get(deployment_name, endpoint_name=endpoint_name)
    if deployment.provisioning_state == 'Succeeded':
        break
    time.sleep(10)

........................................................

In [61]:
endpoint.traffic = {deployment_name: 100}
ml_client.online_endpoints.begin_create_or_update(endpoint)

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


In [75]:
import json

endpoint_host = endpoint.scoring_uri.removesuffix('/score')
keys = ml_client.online_endpoints.get_keys(endpoint_name)
endpoint_details = {
    'host': endpoint_host,
    'key': keys.primary_key
}

with open('online-endpoint.json', 'w') as f:
    f.write(json.dumps(endpoint_details))